[Pwsh 7 adds a bunch of new operators](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_operators?view=powershell-7.3#null-coalescing-operator-), including `Null-coalescing` 

You need at least Pwsh 7+ to run the examples

In [104]:
# The docs start with this syntax:
$list = $Null
${list}?[0]

# I find it cleaner to use:
($list)?[0]

#Example: 
$samples = 'foo', 234, (Get-Date), $null, ''

'# Example that breaks'
$samples | Foreach-Object { 
    $_.GetType()     
} 
'# simple coerce to null'
$samples | Foreach-Object { 
    ($_).GetType()     
}

# Example that breaks

IsPublic IsSerial Name                                     BaseType
-------- -------- ----                                     --------
True     True     String                                   System.Object
True     True     Int32                                    System.ValueType
True     True     DateTime                                 System.ValueType
InvalidOperation: 
Line |
  13 |      $_.GetType()
     |      ~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
True     True     String                                   System.Object
# simple coerce to null
True     True     String                                   System.Object
True     True     Int32                                    System.ValueType
True     True     DateTime                                 System.ValueType
InvalidOperation: 
Line |
  17 |      ($_).GetType()
     |      ~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
True     True   

In [103]:
# using the statement:
$a = gi .

# error: Invoking Method on Null
$a.GetType().Name

# then: this coerces to null
($fake)?.GetType().Name

write-warning 'warning: It doesn''t automatically protect you from invoking another function on $null'
($fake)?.GetType().Name.ToString()

write-warning "You *could* write it like this, it's getting to a point it should be another step"
(($fake)?.GetType().Name)?.ToString()

DirectoryInfo
InvalidOperation: 
Line |
  11 |  ($fake)?.GetType().Name.ToString()
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.


In [108]:


function NewUser { 
   param( [string]$Name, [hashtable]$Config )
   class User { [ValidateNotNull()][string]$Name }

   [User]@{ 
        Name = $Name ?? 'anonymous'         
    }
}

# showing "wrong' samples to see the output

NewUser
NewUser -name 'bob'
NewUser 'sdf' @{ Region = 'Green' }
NewUser @{ Region = 'Green' }



Name
----

bob
sdf
System.Collections.Hashtable



In [ ]:
$samples = 'foo', 234, (get-date), $null, '' 

function shortenTypeNames1z {
    process {
        # expects types or nulls
        [pscustomobject]@{ 
            Original = ($_).GetType() ?? 'a'
            Shorter  = if ($null -eq $_) { 
                "[`u{2400}]" 
            }
            else { 

                Final = ($_.GetType())?.Name
                | Join-String -op '[' -os ']'
            }
         
        }
    }

}



$samples | ShortenTypeNames1z



In [ ]:
$samples = 'foo', 234, (get-date), $null, '' 

function shortenTypeNames1z {
    process {
        # expects types or nulls
        [pscustomobject]@{ 
            Original = ($_).GetType() ?? 'a'
            Shorter  = if ($null -eq $_) { 
                "[`u{2400}]" 
            }
            else { 

                Final = ($_.GetType())?.Name
                | Join-String -op '[' -os ']'
            }
         
        }
    }

}



$samples | ShortenTypeNames1z



In [88]:
$samples = 'foo', 234, (get-date), $null, '' 

function shortenTypeNames2 {
    process {
        # expects types or nulls
        if($null -eq $_) { 'bbad' }
        ($_.GetType())?.Name
        | Join-String -op '[' -os ']'

    }
}


$samples | ShortenTypeNames2

[String]
[Int32]
[DateTime]
bbad
InvalidOperation: 
Line |
   7 |          ($_.GetType())?.Name
     |          ~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
[String]


In [110]:

$testId = 0 ;  $samples = '', 'bob' 
$samples 

function testNewUser {
    process { 
    [pscustomobject]@{ 
        Id    = $testId++
        Name  = ($_)?.ToString().Name ?? '<anonymous>'
        Name2 = ($_)?.ToString()      ?? '<anonymous>'
        TrueEmptyStr        = $_ -is 'string' -and $_.length -eq 0
        IsBlank = [string]::IsNullOrWhiteSpace( $_ )
        IsEmpty = [string]::IsNullOrEmpty( $_ )
        TrueNull            = $null -eq $_ 
    }
} }

$samples
| testNewUser
| Format-Table -AutoSize
        #TrueNull = if($null -eq $_) { '[true ␀]' }/

        #if('' -eq $_ -and $_ -is 'string') { '[true EmptyStr]' }
        #//TrueNull = if($null -eq $_) { '[true ␀]'


bob

bob


In [111]:
import-module pansies
import-module Ninmonkey.Console
# cannot get it to fallback in these expresssions
# even though it reliably does it in func. Maybe its
# that $foo didnt exist, but $_ was?
# or, maybe the () evaluation changes somethiing
hr
$null -eq ($Foo.Bar)?.FakeFunc().Name    # true
($Foo.Bar)?.FakeFunc().Name ?? 'fall'    # fall
hr
$null -eq ($Foo.Bar)?.FakeFunc().Name    #true
($Foo.Bar)?.FakeFunc() ?? 'fall'         # fall


($_)?.ToString().Name ?? '<anonymous>'

($_)?.ToString() ?? '<anonymous>'

hr
$null -eq ($_)?.ToString()          # true
($_)?.ToString() ?? 'fall'

$null -eq ($_)?.ToString().Name     # true
($_)?.ToString().Name ?? 'fall'

Import-Module: The specified module 'pansies' was not loaded because no valid module file was found in any module directory.
Import-Module: 
Line |
   2 |  import-module Ninmonkey.Console
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The required module 'Pansies' is not loaded. Load the module or remove the module from 'RequiredModules' in the file 'C:\Users\cppmo_000\SkyDrive\Documents\2021\powershell\My_Github\Ninmonkey.Console\Ninmonkey.Console.psd1'.
hr: 
Line |
   7 |  hr
     |  ~~
     | The term 'hr' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
True
fall
hr: 
Line |
  10 |  hr
     |  ~~
     | The term 'hr' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
True
fall
<anonymous>
<anonymous>


<!-- buffer dump



import-module pansies
import-module Ninmonkey.Console
# cannot get it to fallback in these expresssions
# even though it reliably does it in func. Maybe its
# that $foo didnt exist, but $_ was?
# or, maybe the () evaluation changes somethiing
hr
$null -eq ($Foo.Bar)?.FakeFunc().Name    # true
($Foo.Bar)?.FakeFunc().Name ?? 'fall'    # fall
hr
$null -eq ($Foo.Bar)?.FakeFunc().Name    #true
($Foo.Bar)?.FakeFunc() ?? 'fall'         # fall


($_)?.ToString().Name ?? '<anonymous>'

($_)?.ToString() ?? '<anonymous>'

hr
$null -eq ($_)?.ToString()          # true
($_)?.ToString() ?? 'fall'

$null -eq ($_)?.ToString().Name     # true
($_)?.ToString().Name ?? 'fall'











 -->